In [18]:
from pyspark.sql import SparkSession
import os
import sys
import pymysql,pyodbc
from pyspark.sql.functions import *
import pandas as pd
from IPython.display import display, HTML
from sqlalchemy import create_engine



In [9]:
spark = SparkSession.builder.master("local").appName("ETL_Pipeline").enableHiveSupport().getOrCreate()
spark

In [3]:
# DEFINE THE DATABASE CREDENTIALS

src_credentials = {}
src_credentials["host"]='localhost'
src_credentials["user"]='root'
src_credentials["password"] = 'root'
src_credentials["db"]='errorlog'

In [ ]:
#import needed libraries
from sqlalchemy import create_engine
import pyodbc
import pandas as pd
import os

#get password from environmnet var
pwd = os.environ['PGPASS']
uid = os.environ['PGUID']
#sql db details
driver = "{SQL Server Native Client 11.0}"
server = "haq-PC"
database = "AdventureWorksDW2019;"

#extract data from sql server
def extract():
    try:
        src_conn = pyodbc.connect('DRIVER=' + driver + ';SERVER=' + server + '\SQLEXPRESS' + ';DATABASE=' + database + ';UID=' + uid + ';PWD=' + pwd)
        src_cursor = src_conn.cursor()
        # execute query
        src_cursor.execute(""" select  t.name as table_name
        from sys.tables t where t.name in ('DimProduct','DimProductSubcategory','DimProductSubcategory','DimProductCategory','DimSalesTerritory','FactInternetSales') """)
        src_tables = src_cursor.fetchall()
        for tbl in src_tables:
            #query and load save data to dataframe
            df = pd.read_sql_query(f'select * FROM {tbl[0]}', src_conn)
            print("T")
            print("no of rows ",len(df))
            load(df, tbl[0])
    except Exception as e:
        print("Data extract error: " + str(e))
    finally:
        src_conn.close()

#load data to postgres
def load(df, tbl):
    try:
        rows_imported = 0
        engine = create_engine(f'postgresql://{uid}:{pwd}@{server}:5432/AdventureWorks')
        print(f'importing rows {rows_imported} to {rows_imported + len(df)}... for table {tbl}')
        # save df to postgres
        df.to_sql(f'stg_{tbl}', engine, if_exists='replace', index=False)
        rows_imported += len(df)
        # add elapsed time to final print out
        print("Data imported successful")
    except Exception as e:
        print("Data load error: " + str(e))

try:
    #call extract function
    extract()
except Exception as e:
    print("Error while extracting data: " + str(e))

In [53]:
# PYTHON FUNCTION TO CONNECT TO THE MYSQL DATABASE

def extract():
    try:
        # GET THE CONNECTION OBJECT (ENGINE) FOR THE DATABASE 
        
        conn = pymysql.connect(**src_credentials)
        cur = conn.cursor()
        
        # cur.execute("select * from errorlog.error_logs limit 100 OFFSET 100")
        # output = cur.fetchall()
        
        # finding the list of columns of table
        # num_fields = len(cur.description)
        # field_names = [i[0] for i in cur.description]
        
        # crea# te the response into a DataFrame
        # df = pd.DataFrame(output,columns =field_names )
        # display(HTML(df.to_html()))
        cur.execute("SHOW TABLES;")
        table_list = cur.fetchall()
        print(table_list)
        for table in table_list:
            df = pd.read_sql_query(f"SELECT * from {table[0]}",conn)
            print("Table name ",table[0])
            print("no of rows ",len(df))
            load(df,table[0])
        tables_list = pd.DataFrame(cur.fetchall(),columns=[i[0] for i in cur.description])
        
        # print(display(HTML(tables_list.to_html())))
      
        # To close the connection
        
        print(f"Connection to the {src_credentials['host']} for user {src_credentials['user']} created successfully.")
    except Exception as ex:
        print("Connection could not be made due to the following error: \n", ex)
    finally:
        conn.close()
extract()

(('defaulter_agents_report',), ('err_log',), ('error_logs',), ('host_ping',), ('it_asset_err_log',), ('it_asset_error_freq',), ('it_asset_error_logs',), ('it_asset_host_ping',), ('it_asset_log_files',), ('ita_err_logs',), ('ita_example',), ('log_files',), ('mqtt_logs',), ('restart_logs',), ('update_logs',), ('version_log',))
Table name  defaulter_agents_report
no of rows  22
Connected to SQL Server successfully!
stg_defaulter_agents_report
Connection closed.
Connection could not be made due to the following error: 
 (pyodbc.InterfaceError) ('IM002', '[IM002] [Microsoft][ODBC Driver Manager] Data source name not found and no default driver specified (0) (SQLDriverConnect)')
(Background on this error at: https://sqlalche.me/e/20/rvf5)


In [52]:
# Set the connection details
# def load(df, table):
#     server_name = "VIBHAV-CV"
#     database_name = "errorlog_from_mysql"

#     # Create the connection string
#     connection_string = f"mssql+pyodbc://{server_name}/{database_name}?driver=ODBC+Driver+17+for+SQL+Server?trusted_connection=yes"

#     try:
#         # Establish the connection
#         engine = create_engine(connection_string)

#         print("Connected to SQL Server successfully!")

#         # Execute SQL queries or perform other operations
#         table_name = f'stg_{table}'
#         print(table_name)
#         #print(df)
#         with engine.connect() as connection:
#             df.to_sql(table_name, con=connection, if_exists='replace', index=False, schema=None)
#             rows_imported += len(df)
#             print(f"stg_{table} is loaded with {len(df)} rows")

#     except pyodbc.Error as e:
#         print(f"Error connecting to SQL Server: {e}")
#     finally:
#         print("Connection closed.")

# Set the connection details
def load(df,table):
    server_name = "VIBHAV-CV"
    database_name = "errorlog_from_mysql"

    # Create the connection string
    #connection_string = f"Driver={{SQL Server}};Server={server_name};Database={database_name};Trusted_Connection=yes;"
    connection_string = f'mssql+pyodbc://{server_name}/{database_name}?driver=ODBC+Driver+17+for+SQL+Server?trusted_connection=yes'

    try:
        # Establish the connection
        # connection = pyodbc.connect(connection_string)
        engine = create_engine(connection_string)
        connection = engine.connect()
        
        print("Connected to SQL Server successfully!")

        # Execute SQL queries or perform other operations
        # cursor = connection.cursor()
        # tables = cursor.tables(tableType='TABLE')
#         table_name = f'stg_{table}'
#         print(table_name)
#         #print(df)
#         df.to_sql(table_name,con=connection,schema=None, if_exists='replace', index=False)
#         rows_imported += len(df)
#         print(f"stg_{table} is loaded with {len(df)} rows")
        # Close the connection
        
        
    except pyodbc.Error as e:
        print(f"Error connecting to SQL Server: {e}")
    finally:
        connection.close()
        print("Connection closed.")
